In [88]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
df1 = pd.read_csv("User Preference Car Sport.csv")
df1.head()

,User ID;Nama_Lengkap;Jenis_Kelamin;Pekerjaan;Usia;Kota_Asal;Car Make;Car Model;Rating;;;
0,1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...
1,1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...
2,1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...
3,1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...
4,1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...


In [90]:
df2 = pd.read_csv("list_cars2.csv")
df2.head()

,Car Make,Car Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,7.0,55.0


In [91]:
# Gabungkan Dataset
merged_df = pd.concat([df1, df2], axis=0)

# Reset ulang indeks
merged_df.reset_index(drop=True, inplace=True)

print("Merged DataFrame secara Vertikal:")
print(merged_df)
# Cetak daftar kolom dalam DataFrame
print(merged_df.columns)

Merged DataFrame secara Vertikal:
     User ID;Nama_Lengkap;Jenis_Kelamin;Pekerjaan;Usia;Kota_Asal;Car Make;Car Model;Rating;;;  \
0     1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...                                         
1     1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...                                         
2     1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...                                         
3     1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...                                         
4     1;Glen Choirul Fais;Laki-laki;Mahasiswa;22;Ban...                                         
...                                                 ...                                         
2252                                                NaN                                         
2253                                                NaN                                         
2254                                                NaN                                      

In [92]:
# Info tentang dataset, termasuk tipe data dan non-null counts
print("Informasi dataset:")
print(merged_df.info())

# Statistik deskriptif untuk setiap kolom numerik
print("\nStatistik deskriptif untuk kolom numerik:")
print(merged_df.describe())

# Cetak daftar kolom dalam dataset
print("\nDaftar kolom dalam dataset:")
print(merged_df.columns)

Informasi dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2257 entries, 0 to 2256
Data columns (total 21 columns):
 #   Column                                                                                    Non-Null Count  Dtype  
---  ------                                                                                    --------------  -----  
 0   User ID;Nama_Lengkap;Jenis_Kelamin;Pekerjaan;Usia;Kota_Asal;Car Make;Car Model;Rating;;;  198 non-null    object 
 1   Car Make                                                                                  2059 non-null   object 
 2   Car Model                                                                                 2059 non-null   object 
 3   Price                                                                                     2059 non-null   float64
 4   Year                                                                                      2059 non-null   float64
 5   Kilometer                           

In [93]:
# i) Mencari dan Memproses Data yang Hilang
missing_data = merged_df.isnull().sum()
print("Data yang Hilang:\n", missing_data)

# ii) Nilai Data Campuran/Tipe Data yang Tidak Cocok
mixed_data = merged_df.applymap(type).nunique()
print("\nTipe Data Campuran:\n", mixed_data)

# iii) Pencarian Pencilan pada Data
# Menghitung z-score untuk kolom numerik
numeric_cols = merged_df.select_dtypes(include=['int', 'float']).columns
z_scores = (merged_df[numeric_cols] - merged_df[numeric_cols].mean()) / merged_df[numeric_cols].std()

# Menandai baris yang memiliki nilai z-score > 3 atau < -3 sebagai outlier
outliers = merged_df[(z_scores > 3) | (z_scores < -3)]
print("\nData Pencilan:\n", outliers)

Data yang Hilang:
 User ID;Nama_Lengkap;Jenis_Kelamin;Pekerjaan;Usia;Kota_Asal;Car Make;Car Model;Rating;;;    2059
Car Make                                                                                     198
Car Model                                                                                    198
Price                                                                                        198
Year                                                                                         198
Kilometer                                                                                    198
Fuel Type                                                                                    198
Transmission                                                                                 198
Location                                                                                     198
Color                                                                                        198
Owner      

In [94]:
# Collaborative Filtering
# Buat User-Item Matrix
# user_item_matrix = merged_df.pivot_table(index='User ID', columns='Car Make', values='Rating', fill_value=0)

# Jika 'Rating' ada dalam dataset
# print(user_item_matrix)

In [95]:
# Hitung Similarity antar pengguna
# user_similarity = cosine_similarity(user_item_matrix, user_item_matrix)

In [96]:
# Content-Based Filtering
# Combine information from multiple columns into one text column
merged_df['Car Features'] = merged_df['Car Make'] + ' ' + merged_df['Car Model'] + ' ' + merged_df['Year'].astype(str) + ' ' + merged_df['Color'] + ' ' + merged_df['Transmission'] + ' ' + merged_df['Fuel Type']

# Drop rows with NaN values in 'Car Features' column
merged_df = merged_df.dropna(subset=['Car Features'])

# Initialize TF-IDF vectorizer
# Ekstraksi Fitur
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(merged_df['Car Features'])

# Fit and transform the text data
car_features_tfidf = tfidf.fit_transform(merged_df['Car Features'].astype(str))

# Print feature names (words in the vocabulary)
print("Vocabulary:", tfidf.get_feature_names_out())

Vocabulary: ['0d' '0l' '110' '143' '143bhp' '16' '177bhp' '180' '1988' '1996' '1l'
 '200' '2000' '2002' '2004' '2006' '2007' '2008' '2009' '200d' '2010'
 '2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019' '2020'
 '2021' '2022' '20d' '220' '220d' '250' '250xl' '280' '2l' '2wd' '30'
 '300' '300d' '300h' '30d' '320d' '320i' '330i' '35' '350' '350d' '40'
 '400' '400d' '43' '45' '450' '488' '4matic' '4wd' '4x2' '4x4' '500'
 '500l' '520d' '525d' '530d' '530i' '55' '560' '5l' '610' '630d' '6s'
 '718' '730ld' '75' '800' '85' '8s' '8v' '911' '99' '9l' 'a3' 'a4' 'a6'
 'a7' 'a8' 'abs' 'ac' 'accent' 'accord' 'active' 'activity' 'adventure'
 'ags' 'airbag' 'alcazar' 'alloy' 'allspace' 'alpha' 'altis' 'alto'
 'altroz' 'alturas' 'amaze' 'ambiente' 'ambition' 'ameo' 'amg' 'amt'
 'anniversary' 'aqua' 'aspire' 'asta' 'astor' 'at' 'audi' 'aura'
 'automatic' 'avantgarde' 'avn' 'awd' 'ax' 'b180' 'baleno' 'base' 'beat'
 'beige' 'benz' 'bhp' 'bl' 'black' 'blue' 'blueefficiency' 'bmw' 'bolero'
 'b

In [97]:
# Hitung Similarity antar mobil
car_similarity = cosine_similarity(car_features_tfidf, tfidf_matrix)
print(merged_df.columns)

Index(['User ID;Nama_Lengkap;Jenis_Kelamin;Pekerjaan;Usia;Kota_Asal;Car Make;Car Model;Rating;;;',
       'Car Make', 'Car Model', 'Price', 'Year', 'Kilometer', 'Fuel Type',
       'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine',
       'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height',
       'Seating Capacity', 'Fuel Tank Capacity', 'Car Features'],
      dtype='object')


In [98]:
# Buat fungsi rekomendasi
def recommend_car(user_id, car_similarity=car_similarity, df=merged_df, n=5):
    user_ratings = df[df['User ID'] == user_id]
    
    if user_ratings.empty:
        return "User ID not found in the dataset"
    
    # Gabungkan preferensi mobil pengguna
    user_features = user_ratings['Car Features'].tolist()
    user_features = ' '.join(user_features)
    
    # Hitung similarity dengan preferensi pengguna
    user_tfidf = tfidf.transform([user_features])
    sim_scores = list(car_similarity(user_tfidf, tfidf_matrix).flatten())
    
    # Urutkan mobil berdasarkan similarity scores
    car_indices = sorted(range(len(sim_scores)), key=lambda i: sim_scores[i], reverse=True)[:n]
    
    # Return top n mobil yang direkomendasikan
    return df.iloc[car_indices]

In [99]:
# Menampilkan User ID yang ada dalam dataset
user_ids = merged_df['User ID'].unique()
print("User IDs in the dataset:")
print(user_ids)

KeyError: 'User ID'

In [ ]:
# Contoh penggunaan
user_id = 1  # ID pengguna yang ada dalam dataset
if user_id in merged_df['User ID'].unique():
    recommended_cars = recommend_car(user_id, car_similarity=car_similarity, df=merged_df, n=5)
    print(recommended_cars)
else:
    print("User ID not found in the dataset")

User ID not found in the dataset
